<h1 align="center">Long Short-Term Memory (LSTM) Networks </h1> 

### The Problem of Long-Term Dependencies

One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task, such as using previous video frames might inform the understanding of the present frame. If RNNs could do this, they’d be extremely useful. But can they? It depends.

Sometimes, we only need to look at recent information to perform the present task. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “the clouds are in the sky,” we don’t need any further context – it’s pretty obvious the next word is going to be sky. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.

But there are also cases where we need more context. Consider trying to predict the last word in the text “I grew up in France… I speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of France, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large. Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.

![title](../images/long_term_dependencies.jpg)

### LSTM Networks

LSTM are a special kind of RNN, capable of learning long-term dependencies. LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn! All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer (figure below, left):

In [4]:
function mlp1_rnn(param, hₜ₋, xₜ)
    input  = hcat(hₜ₋, xₜ)
    hₜ = tanh(input * param[1] .+ param[2])
    return (hₜ, xₜ)
end

rnn_tanh_layer (generic function with 1 method)

LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way (figure below, right):

![title](../images/RNN_vs_LSTM.jpg)

In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pink circles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers. Lines merging denote concatenation, while a line forking denote its content being copied and the copies going to different locations.

### The Core Idea Behind LSTMs

The key to LSTMs is the cell state $C$, the horizontal line running through the top of the diagram: entering from the top left as $C_{t-1}$ and exiting from the top right as $C_t$.

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions (vector addition and multiplication). It’s very easy for information to just flow along it unchanged. The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates. Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation (first, second, and fourth learned layers). The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”

An LSTM has three of these gates, to protect and control the cell state.

### Step-by-Step LSTM Walk Through

The first step in our LSTM is to decide what information we’re going to throw away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer” and it's denoted as $f$. It looks at $h_{t-1}$ and $x_t$, and outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.” Formally:

\begin{equation}
f_t = \sigma\big(W_f\, [h_{t-1}\,\,x_t] + b_f\big)
\end{equation}


Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

The next step is to decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” (denoted by $i$) decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values (denoted by $C$), $\tilde{C}_t$, that could be added to the state. In the next step, we’ll combine these two to create an update to the state. Formally:

\begin{equation}
i_t = \sigma\big(W_i\, [h_{t-1}\,\,x_t] + b_i\big)\hspace{.7cm}
\end{equation}
\begin{equation}
\tilde{C}_t = \tanh\big(W_C\, [h_{t-1}\,\,x_t] + b_C\big)
\end{equation}


In the example of our language model, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.

It’s now time to update the old cell state, $C_{t−1}$, into the new cell state $C_t$. The previous steps already decided what to do, we just need to actually do it. We multiply the old state by $f_t$, forgetting the things we decided to forget earlier. Then we add $i_t∗\tilde{C}_t$. This is the new candidate values, scaled by how much we decided to update each state value. Formally:

\begin{equation}
C_t = f_t*C_{t-1} + i_t*\tilde{C}
\end{equation}


In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output and denote the output by $o$. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to. Formally:

\begin{equation}
o_t = \sigma\big(W_o\, [h_{t-1}\,\,x_t] + b_o]\big)
\end{equation}
\begin{equation}
h_t = o_t * \tanh C_t\hspace{1.8cm}
\end{equation}


For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.

In summary, we have the following equations for a single-layer LSTM network:

\begin{equation}
\begin{aligned}
f_t &= \sigma\big(W_f\, [h_{t-1}\,\,x_t] + b_f\big) & (1) &\, \text{Forget gate} \\
i_t &= \sigma\big(W_i\, [h_{t-1}\,\,x_t] + b_i\big) & (2) &\, \text{Input gate} \\
\tilde{C}_t &= \tanh\big(W_C\, [h_{t-1}\,\,x_t] + b_C\big) & (3) &\, \text{Cell candidate} \\
C_t &= f_t*C_{t-1} + i_t*\tilde{C} & (4) &\, \text{New cell}\\
o_t &= \sigma\big(W_o\, [h_{t-1}\,\,x_t] + b_o]\big) & (5) &\, \text{Output gate} \\
h_t &= o_t * \tanh C_t & (6) &\, \text{New output} 
\end{aligned}
\end{equation}

Let us consider the size of all the variables involved. The size of the input vector $x$ is denoted as $I$, while the size of $h$ and $C$ are both denoted as $H$, therefore the concatenated input $[h_{t-1}\,\,x_t]$ has a size $H+I$. In practice, the weight matrices $W_f, W_i, W_C$ and $W_o$ are combined into a single weight matrix $W$, therefore the size of the combined matrix $W$ is $(H+I)\times 4H$ and the size of $b$ is $H$. This is only true for a single time-step; for multiple time-steps we have I=H, thefore the size of the matrix $W$ is in general $2H\times 4H$.

Taking into consideration that a single lstm layer has three outputs at any given time ($h_t$ moving forward and $h_t$, $C_t$ delayed and feed to itself at a later time), let's now see the implementation in Julia: 

In [1]:
function lstm(param, stateₜ₋, xₜ)
    
    weight, bias = param
    hₜ₋, Cₜ₋      = stateₜ₋
    input        = hcat(hₜ₋, xₜ)
    
    H            = size(hₜ₋, 2)
    pre_gates    = input * weight .+ bias
    

    fₜ      = sigm.(pre_gates[:, 1:H])       # (1) forget gate
    iₜ      = sigm.(pre_gates[:, 1 + H:2H])  # (2) input gate
    oₜ      = sigm.(pre_gates[:, 1 + 2H:3H]) # (5) output gate
    changeₜ = tanh.(pre_gates[:, 1 + 3H:4H]) # (3) cell candidate
    
    Cₜ = fₜ .* Cₜ₋ + iₜ .* changeₜ             # (4) new cell 
    hₜ = oₜ .* tanh.(Cₜ)                      # (6) new output
    
    return (hₜ, Cₜ)

end

lstm (generic function with 1 method)

The ouput (readout, or prediction layer) is given by:

\begin{equation}
y = ph 
\end{equation}

where $P$ denotes the weights of the prediction matrix. Let us denote the size of the output vector $y$ as $O$ such that $p$ has size $H\times O$.  